In [68]:
import matplotlib.pyplot as plt
from IPython.display import Image, display, clear_output
import numpy as np
%matplotlib nbagg
%matplotlib inline
import seaborn as sns
import pandas as pd
sns.set_style("whitegrid")
import pickle
import torch
from sklearn.model_selection import train_test_split
from torch import nn, optim
import torch.nn.functional as F
import copy

RANDOM_SEED = 56
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [2]:
# Load data
df0 = pickle.load( open( "data/aligned_GM-7792_DRD-d6b1cf27-41ed-43b6-8050-2068ef941a0aDRD-d6b1cf27-41ed-43b6-8050-2068ef941a0a_GM-7792_pass-0_cleaned.pickle", "rb" ) )
df1 = pickle.load( open( "data/aligned_GM-7792_DRD-d6b1cf27-41ed-43b6-8050-2068ef941a0aDRD-d6b1cf27-41ed-43b6-8050-2068ef941a0a_GM-7792_pass-1_cleaned.pickle", "rb" ) )

In [3]:
df = df0.append(df1, ignore_index=True)

In [4]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,arff2pandas

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.29.0

numpy      : 1.21.4
pandas     : 1.3.4
torch      : 1.10.0
arff2pandas: 1.0.1



In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [7]:
df.head()

,IRI_mean,IRI_sequence,GM.TS_or_Distance,GM.T,GM.lat_int,GM.lon_int,GM.acc.xyz.z,GM.acc.xyz.y,GM.acc.xyz.x,GM.obd.spd_veh.value
1697,3.835,"[1.8601703035, 3.2504808755, 5.053189786, 4.01...","[2021-04-22 01:08:16.465000+00:00, 2021-04-22 ...","[acc.xyz, acc.xyz, acc.xyz, acc.xyz, acc.xyz, ...","[55.66611856549521, 55.666118079872206, 55.666...","[12.567482945686901, 12.567481884984025, 12.56...","[0.9883, 0.9883, 0.9883, 0.9766, 0.9766, 0.976...","[-0.0508, -0.0508, -0.0508, -0.0586, -0.0586, ...","[0.1367, 0.1367, 0.1367, 0.1367, 0.1367, 0.136...","[56.980000000000004, 56.95, 56.92, 56.9, 56.9,..."
3785,1.75299,"[0.9589948954999999, 1.864791327, 1.6323926260...","[2021-04-22 02:37:03.444000+00:00, 2021-04-22 ...","[acc.xyz, acc.xyz, obd.spd_veh, acc.xyz, acc.x...","[55.65223110094637, 55.65223118611987, 55.6522...","[12.495351993690852, 12.495351425867508, 12.49...","[0.9727, 0.9727, 0.9727, 0.9727, 0.9844, 0.984...","[-0.082, -0.082, -0.082, -0.082, -0.0781, -0.0...","[0.1563, 0.1563, 0.1563, 0.1563, 0.1602, 0.160...","[30.43, 30.400000000000002, 30.38, 30.58000000..."
895,2.99175,"[2.567165461, 3.065463918, 2.1012670425, 1.499...","[2021-04-22 01:48:01.859000+00:00, 2021-04-22 ...","[acc.xyz, acc.xyz, obd.spd_veh, acc.xyz, acc.x...","[55.72094770915033, 55.72094811111111, 55.7209...","[12.544176196078432, 12.544176666666667, 12.54...","[0.9766, 0.9805, 0.9805, 0.9805, 0.9805, 0.980...","[-0.0781, -0.0664, -0.0664, -0.0664, -0.0664, ...","[0.1328, 0.1328, 0.1328, 0.1328, 0.1328, 0.132...","[47.45, 47.44, 47.43, 47.43, 47.42, 47.42, 47...."
1368,3.433299,"[6.6389056895, 3.828659857, 2.5453005429999997...","[2021-04-22 01:56:23.880000+00:00, 2021-04-22 ...","[acc.xyz, acc.xyz, acc.xyz, obd.spd_veh, acc.x...","[55.701853409240925, 55.70185274917492, 55.701...","[12.592269062706272, 12.592269155115511, 12.59...","[1.0313, 1.0313, 1.0313, 1.0313, 1.0313, 1.031...","[-0.0352, -0.0352, -0.0352, -0.0352, -0.0352, ...","[0.2305, 0.2305, 0.2305, 0.2305, 0.2305, 0.230...","[41.74, 42.02, 41.74, 41.74, 41.75, 41.77, 41...."
757,7.488447,"[8.930436514, 2.9415233054999996, 1.657743437,...","[2021-04-22 01:42:27.050000+00:00, 2021-04-22 ...","[acc.xyz, acc.xyz, acc.xyz, acc.xyz, acc.xyz, ...","[55.7115501559633, 55.71155024770642, 55.71155...","[12.531196467889908, 12.531196743119265, 12.53...","[1.0508, 1.0508, 1.0508, 1.0508, 1.0352, 1.035...","[0.043, 0.043, 0.043, 0.043, 0.0391, 0.0391, 0...","[0.1758, 0.1758, 0.1758, 0.1758, 0.1758, 0.175...","[35.26, 35.34, 35.42, 35.51, 35.59, 35.65, 35...."


# Pad with initial zeroes

In [25]:
ts_max_len = max(df['GM.acc.xyz.z'].apply(len))
padded_ts = []

for index, ls in df['GM.acc.xyz.z'].items():
    padded_ts.append(np.concatenate(([0] * (ts_max_len - len(ls)), ls)))
    
s1 = pd.Series(padded_ts, index=df.index, name='GM.acc.xyz.z')

In [30]:
df['GM.acc.xyz.z'] = s1

In [27]:
#shuffle df
df = df.sample(frac=1.0)
df.shape

(3830, 11)

In [45]:
iri_threshold = 5
normal_df = df['GM.acc.xyz.z'][df['IRI_mean'] <= iri_threshold]
normal_df.shape

(3410,)

In [46]:
anomaly_df = df['GM.acc.xyz.z'][df['IRI_mean'] >= iri_threshold]
anomaly_df.shape

(420,)

In [48]:
anomaly_df = pd.DataFrame(anomaly_df.tolist())

In [50]:
normal_df = pd.DataFrame(normal_df.tolist())

In [55]:
train_df, val_df = train_test_split(
  normal_df,
  test_size=0.15,
  random_state=RANDOM_SEED
)

val_df, test_df = train_test_split(
  val_df,
  test_size=0.33, 
  random_state=RANDOM_SEED
)

# convert to tensors

In [56]:
def create_dataset(df):

  sequences = df.astype(np.float32).to_numpy().tolist()

  dataset = [torch.tensor(s).unsqueeze(1).float() for s in sequences]

  n_seq, seq_len, n_features = torch.stack(dataset).shape

  return dataset, seq_len, n_features

In [57]:
train_dataset, seq_len, n_features = create_dataset(train_df)
val_dataset, _, _ = create_dataset(val_df)
test_normal_dataset, _, _ = create_dataset(test_df)
test_anomaly_dataset, _, _ = create_dataset(anomaly_df)

In [60]:
class Encoder(nn.Module):

  def __init__(self, seq_len, n_features, embedding_dim=64):
    super(Encoder, self).__init__()

    self.seq_len, self.n_features = seq_len, n_features
    self.embedding_dim, self.hidden_dim = embedding_dim, 2 * embedding_dim

    self.rnn1 = nn.LSTM(
      input_size=n_features,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )
    
    self.rnn2 = nn.LSTM(
      input_size=self.hidden_dim,
      hidden_size=embedding_dim,
      num_layers=1,
      batch_first=True
    )

  def forward(self, x):
    x = x.reshape((1, self.seq_len, self.n_features))

    x, (_, _) = self.rnn1(x)
    x, (hidden_n, _) = self.rnn2(x)

    return hidden_n.reshape((self.n_features, self.embedding_dim))

In [61]:
class Decoder(nn.Module):

  def __init__(self, seq_len, input_dim=64, n_features=1):
    super(Decoder, self).__init__()

    self.seq_len, self.input_dim = seq_len, input_dim
    self.hidden_dim, self.n_features = 2 * input_dim, n_features

    self.rnn1 = nn.LSTM(
      input_size=input_dim,
      hidden_size=input_dim,
      num_layers=1,
      batch_first=True
    )

    self.rnn2 = nn.LSTM(
      input_size=input_dim,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )

    self.output_layer = nn.Linear(self.hidden_dim, n_features)

  def forward(self, x):
    x = x.repeat(self.seq_len, self.n_features)
    x = x.reshape((self.n_features, self.seq_len, self.input_dim))

    x, (hidden_n, cell_n) = self.rnn1(x)
    x, (hidden_n, cell_n) = self.rnn2(x)
    x = x.reshape((self.seq_len, self.hidden_dim))

    return self.output_layer(x)

In [62]:
class RecurrentAutoencoder(nn.Module):

  def __init__(self, seq_len, n_features, embedding_dim=64):
    super(RecurrentAutoencoder, self).__init__()

    self.encoder = Encoder(seq_len, n_features, embedding_dim).to(device)
    self.decoder = Decoder(seq_len, embedding_dim, n_features).to(device)

  def forward(self, x):
    x = self.encoder(x)
    x = self.decoder(x)

    return x

In [63]:
model = RecurrentAutoencoder(seq_len, n_features, 128)
model = model.to(device)

In [64]:
def train_model(model, train_dataset, val_dataset, n_epochs):
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
  criterion = nn.L1Loss(reduction='sum').to(device)
  history = dict(train=[], val=[])

  best_model_wts = copy.deepcopy(model.state_dict())
  best_loss = 10000.0
  
  for epoch in range(1, n_epochs + 1):
    model = model.train()

    train_losses = []
    for seq_true in train_dataset:
      optimizer.zero_grad()

      seq_true = seq_true.to(device)
      seq_pred = model(seq_true)

      loss = criterion(seq_pred, seq_true)

      loss.backward()
      optimizer.step()

      train_losses.append(loss.item())

    val_losses = []
    model = model.eval()
    with torch.no_grad():
      for seq_true in val_dataset:

        seq_true = seq_true.to(device)
        seq_pred = model(seq_true)

        loss = criterion(seq_pred, seq_true)
        val_losses.append(loss.item())

    train_loss = np.mean(train_losses)
    val_loss = np.mean(val_losses)

    history['train'].append(train_loss)
    history['val'].append(val_loss)

    if val_loss < best_loss:
      best_loss = val_loss
      best_model_wts = copy.deepcopy(model.state_dict())

    print(f'Epoch {epoch}: train loss {train_loss} val loss {val_loss}')

  model.load_state_dict(best_model_wts)
  return model.eval(), history

In [ ]:
model, history = train_model(
  model, 
  train_dataset, 
  val_dataset, 
  n_epochs=150
)